# NLTK Corpus Analysis with Gensim's LDA Model 

## Preparation
First of all, you need to import necessary libraries (with pip command).
* nltk
* gensim
* pyLDAvis

In [315]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

After installing the dependencies, you need to download the following datasets.

In [316]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("semcor")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Datasets
Load the corpus from NLTK package.

In [317]:
from nltk.corpus import semcor as corpus

Let us check out the content of the corpus.

In [318]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:100000]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")
    
print("")
print("")
print("Total number of documents:",len(corpus.fileids()))

The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced `` no evidence '' that any irregularities took place  
. The jury further said in term end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves  
the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September October term jury  
had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won  
by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the  
widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that  
many of Georgia 's registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislators act

You can train the model with first K number of documents or all documents.

In [319]:
# All documents
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:10])
print ("")
print("num of docs:", len(docs))

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...], ['Committee', 'approval', 'of', 'Gov.', 'Price', ...], ['The', 'Orioles', 'tonight', 'retained', 'the', ...], ['A', 'Texas', 'halfback', 'who', 'does', "n't", ...], ['Rookie', 'Ron', 'Nischwitz', 'continued', 'his', ...], ['Nick', 'Skorich', ',', 'the', 'line', 'coach', 'for', ...], ['If', 'the', 'Cardinals', 'heed', 'Manager', 'Gene', ...], ['Sizzling', 'temperatures', 'and', 'hot', 'summer', ...], ['The', 'nuclear', 'war', 'is', 'already', 'being', ...], ['It', 'is', 'not', 'news', 'that', 'Nathan', ...]]

num of docs: 352


## Data preprocessing
First, let us define some stopwords. Here we consider English stopwords from the NLTK package and some noises that may affect our LDA analysis result.  
(Optional) Try to ignore numbers and words through regular expression.

In [320]:
# English stopwords defined by the NLTK package.
from nltk.corpus import stopwords
en_stop = stopwords.words("english")

# Ignore noises that might affect our result.
en_stop = ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<","$","''","*","!"]\
+["0","1","2","3","4","5","6","7","8","9","10","87","50","100","31","29","13","637","71","30","74","1119","18","1913","1923","1937","1961","1962","87","402"]\
+["f","a's" ,"mr.", "able" , "about" , "above" , "according" , "accordingly" , "across" , "actually" , "after" , "afterwards" , "again" , "against" , "ain't" , "all" , "allow" , "allows" , "almost" , "alone" , "along" , "already" , "also" , "although" , "always" , "am" , "among" , "amongst" , "an" , "and" , "another" , "any" , "anybody" , "anyhow" , "anyone" , "anything" , "anyway" , "anyways" , "anywhere" , "apart" , "appear" , "appreciate" , "appropriate" , "are" , "aren't" , "around" , "as" , "aside" , "ask" , "asking" , "associated" , "at" , "available" , "away" , "awfully" , "be" , "became" , "because" , "become" , "becomes" , "becoming" , "been" , "before" , "beforehand" , "behind" , "being" , "believe" , "below" , "beside" , "besides" , "best" , "better" , "between" , "beyond" , "both" , "brief" , "but" , "by" , "c'mon" , "c's" , "came" , "can" , "can't" , "cannot" , "cant" , "cause" , "causes" , "certain" , "certainly" , "changes" , "clearly" , "co" , "com" , "come" , "comes" , "concerning" , "consequently" , "consider" , "considering" , "contain" , "containing" , "contains" , "corresponding" , "could" , "couldn't" , "course" , "currently" , "definitely" , "described" , "despite" , "did" , "didn't" , "different" , "do" , "does" , "doesn't" , "doing" , "don't" , "done" , "down" , "downwards" , "during" , "each" , "edu" , "eg" , "eight" , "either" , "else" , "elsewhere" , "enough" , "entirely" , "especially" , "et" , "etc" , "even" , "ever" , "every" , "everybody" , "everyone" , "everything" , "everywhere" , "ex" , "exactly" , "example" , "except" , "far" , "few" , "fifth" , "first" , "five" , "followed" , "following" , "follows" , "for" , "former" , "formerly" , "forth" , "four" , "from" , "further" , "furthermore" , "get" , "gets" , "getting" , "given" , "gives" , "go" , "goes" , "going" , "gone" , "got" , "gotten" , "greetings" , "had" , "hadn't" , "happens" , "hardly" , "has" , "hasn't" , "have" , "haven't" , "having" , "he" , "he's" , "hello" , "help" , "hence" , "her" , "here" , "here's" , "hereafter" , "hereby" , "herein" , "hereupon" , "hers" , "herself" , "hi" , "him" , "himself" , "his" , "hither" , "hopefully" , "how" , "howbeit" , "however" , "i'd" , "i'll" , "i'm" , "i've" , "ie" , "if" , "ignored" , "immediate" , "in" , "inasmuch" , "inc" , "indeed" , "indicate" , "indicated" , "indicates" , "inner" , "insofar" , "instead" , "into" , "inward" , "is" , "isn't" , "it" , "it'd" , "it'll" , "it's" , "its" , "itself" , "just" , "keep" , "keeps" , "kept" , "know" , "known" , "knows" , "last" , "lately" , "later" , "latter" , "latterly" , "least" , "less" , "lest" , "let" , "let's" , "like" , "liked" , "likely" , "little" , "look" , "looking" , "looks" , "ltd" , "mainly" , "many" , "may" , "maybe" , "me" , "mean" , "meanwhile" , "merely" , "might" , "more" , "moreover" , "most" , "mostly" , "much" , "must" , "my" , "myself" , "name" , "namely" , "nd" , "near" , "nearly" , "necessary" , "need" , "needs" , "neither" , "never" , "nevertheless" , "new" , "next" , "nine" , "no" , "nobody" , "non" , "none" , "noone" , "nor" , "normally" , "not" , "nothing" , "novel" , "now" , "nowhere" , "obviously" , "of" , "off" , "often" , "oh" , "ok" , "okay" , "old" , "on" , "once" , "one" , "ones" , "only" , "onto" , "or" , "other" , "others" , "otherwise" , "ought" , "our" , "ours" , "ourselves" , "out" , "outside" , "over" , "overall" , "own" , "particular" , "particularly" , "per" , "perhaps" , "placed" , "please" , "plus" , "possible" , "presumably" , "probably" , "provides" , "que" , "quite" , "qv" , "rather" , "rd" , "re" , "really" , "reasonably" , "regarding" , "regardless" , "regards" , "relatively" , "respectively" , "right" , "said" , "same" , "saw" , "say" , "saying" , "says" , "second" , "secondly" , "see" , "seeing" , "seem" , "seemed" , "seeming" , "seems" , "seen" , "self" , "selves" , "sensible" , "sent" , "serious" , "seriously" , "seven" , "several" , "shall" , "she" , "should" , "shouldn't" , "since" , "six" , "so" , "some" , "somebody" , "somehow" , "someone" , "something" , "sometime" , "sometimes" , "somewhat" , "somewhere" , "soon" , "sorry" , "specified" , "specify" , "specifying" , "still" , "sub" , "such" , "sup" , "sure" , "t's" , "take" , "taken" , "tell" , "tends" , "th" , "than" , "thank" , "thanks" , "thanx" , "that" , "that's" , "thats" , "the" , "their" , "theirs" , "them" , "themselves" , "then" , "thence" , "there" , "there's" , "thereafter" , "thereby" , "therefore" , "therein" , "theres" , "thereupon" , "these" , "they" , "they'd" , "they'll" , "they're" , "they've" , "think" , "third" , "this" , "thorough" , "thoroughly" , "those" , "though" , "three" , "through" , "throughout" , "thru" , "thus" , "to" , "together" , "too" , "took" , "toward" , "towards" , "tried" , "tries" , "truly" , "try" , "trying" , "twice" , "two" , "un" , "under" , "unfortunately" , "unless" , "unlikely" , "until" , "unto" , "up" , "upon" , "us" , "use" , "used" , "useful" , "uses" , "using" , "usually" , "value" , "various" , "very" , "via" , "viz" , "vs" , "want" , "wants" , "was" , "wasn't" , "way" , "we" , "we'd" , "we'll" , "we're" , "we've" , "welcome" , "well" , "went" , "were" , "weren't" , "what" , "what's" , "whatever" , "when" , "whence" , "whenever" , "where" , "where's" , "whereafter" , "whereas" , "whereby" , "wherein" , "whereupon" , "wherever" , "whether" , "which" , "while" , "whither" , "who" , "who's" , "whoever" , "whole" , "whom" , "whose" , "why" , "will" , "willing" , "wish" , "with" , "within" , "without" , "won't" , "wonder" , "would" , "wouldn't" , "yes" , "yet" , "you" , "you'd" , "you'll" , "you're" , "you've" , "your" , "yours" , "yourself" , "yourselves" , "zero"]\
+["sunday","friday","september","october","million","said","tuesday","wednesday","friday","years","monday","november","jan.","aug.","sept.","go","back","one","year"]\
+[" of ","gov.","w.","add","size","jr.","end","mrs.","m.","see","rep.","b.","d.","13th","yet","n't","want","put","p.","make","say","let","tabb","gop","tom","four"]\
+en_stop

Next, let us define several preprocessing functions.

In [321]:
from nltk.corpus import wordnet as wn # import for lemmatize

def preprocess_word(word, stopwordset):
    
    #1.convert words to lowercase (e.g., Python =>python)
    word=word.lower()
    
    #2.remove ",", ".", and "'s"
    if word in [",",".","'s"]:
        return None
    
    #3.remove stopwords  (e.g., the => (None)) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  (e.g., cooked=>cook)
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    # lemmatized words could be in the stopwords set
    elif lemma in stopwordset: 
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

Let us check out the preprocessing result.

In [322]:
# before
print(docs[0][:10]) 

# after
print(preprocess_documents(docs)[0][:10])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of']
['fulton', 'county', 'grand', 'jury', 'investigation', 'atlanta', 'recent', 'primary', 'election', 'produce']


Next, we need to reshape our documents with the available format for the gensim LDA model.

In [323]:
import gensim
from gensim import corpora

In [324]:
# build the dictionary
dictionary = corpora.Dictionary(preprocess_documents(docs))
# construct the corpus
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

Let us check out the contents of the built dictionary and corpus.

In [325]:
# token2id is the attribute which indicates the mapping between words and dictionary ID

print(dictionary.token2id)

{'1958': 0, 'accept': 1, 'achieve': 2, 'act': 3, 'action': 4, 'additional': 5, 'adjournment': 6, 'adjustment': 7, 'administration': 8, 'administrator': 9, 'afternoon': 10, 'age': 11, 'agree': 12, 'agriculture': 13, 'aid': 14, 'airport': 15, 'ala.': 16, 'allen': 17, 'allot': 18, 'allowance': 19, 'alpharetta': 20, 'alternative': 21, 'ambiguous': 22, 'amendment': 23, 'amicable': 24, 'announce': 25, 'anonymous': 26, 'apparently': 27, 'appoint': 28, 'appointment': 29, 'appraiser': 30, 'approve': 31, 'area': 32, 'arm': 33, 'assistance': 34, 'assistant': 35, 'association': 36, 'atlanta': 37, 'attend': 38, 'attorney': 39, 'audience': 40, 'authorities': 41, 'authority': 42, 'automobile': 43, 'awarding': 44, 'ballot': 45, 'bar': 46, 'barber': 47, 'battle': 48, 'begin': 49, 'belief': 50, 'bellwood': 51, 'berry': 52, 'birth': 53, 'bit': 54, 'blue': 55, 'board': 56, 'body': 57, 'bond': 58, 'bowden': 59, 'bring': 60, 'burden': 61, 'bush': 62, 'byrd': 63, 'caldwell': 64, 'call': 65, 'callan': 66, 'ca

In [326]:
# corpus_ contains words of each document with a list (ID, appear frequency)

# note that there is not the appearing order in the documents, but the order of the dictionary
print(corpus_[0][:10]) 

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 4), (5, 2), (6, 2), (7, 1), (8, 2), (9, 1)]


Let us compare the original document with our preprocessing result that is available for the LDA model.

In [327]:
# before
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])

# after
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

['the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', 'atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
[(37, 1), (108, 1), (151, 1), (165, 1), (196, 1), (211, 1), (249, 1), (262, 1), (340, 1), (360, 1), (392, 1)]


## Training with k = 10, alpha-beta-minimum_probability = 0.1

In [328]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=10,
                                           id2word=dictionary,
                                           passes=50,
                                           alpha=0.1,                 # optional LDA hyperparameter alpha
                                           eta=0.1,                  # optional LDA hyperparameter beta
                                           minimum_probability=0.1    # optional the lower bound of the topic/word generative probability
                                          )

Check out the learned parameters.

In [329]:
# the top num_words of words for each topic (topic ID, the word generative probability for the topic).

topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.008*"man" + 0.005*"head" + 0.005*"time" + 0.005*"house" + 0.004*"eyes" + 0.004*"turn" + 0.004*"thought" + 0.004*"door" + 0.004*"long" + 0.004*"face" + 0.004*"men" + 0.003*"day" + 0.003*"stand" + 0.003*"hear" + 0.003*"side"')
(1, '0.010*"state" + 0.005*"system" + 0.005*"increase" + 0.004*"tax" + 0.004*"**f" + 0.004*"program" + 0.004*"area" + 0.004*"result" + 0.003*"line" + 0.003*"point" + 0.003*"small" + 0.003*"development" + 0.003*"government" + 0.003*"number" + 0.003*"problem"')
(2, '0.004*"man" + 0.004*"time" + 0.003*"state" + 0.003*"house" + 0.002*"president" + 0.002*"good" + 0.002*"men" + 0.002*"police" + 0.002*"county" + 0.002*"home" + 0.002*"people" + 0.002*"long" + 0.002*"found" + 0.002*"thought" + 0.002*"call"')
(3, '0.006*"church" + 0.004*"member" + 0.003*"atom" + 0.003*"du" + 0.002*"general" + 0.002*"pont" + 0.002*"jess" + 0.002*"city" + 0.002*"miss" + 0.002*"wave" + 0.002*"time" + 0.002*"congregation" + 0.002*"life" + 0.002*"curt" + 0.002*"people"')
(4, '0.008*"form" 

In [330]:
# for each document, show the probabilities of topics which beyond the minimum_probability [(topic ID, probability)]

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(1, 0.15732491), (2, 0.8418147)]
document ID 1:[(1, 0.12643097), (2, 0.19939587), (8, 0.6154756)]
document ID 2:[(5, 0.3593886), (9, 0.6398308)]
document ID 3:[(5, 0.47411257), (6, 0.52476287)]
document ID 4:[(5, 0.99904525)]
document ID 5:[(5, 0.9711224)]
document ID 6:[(5, 0.99903)]
document ID 7:[(6, 0.15368669), (7, 0.6589051)]
document ID 8:[(6, 0.72589433), (9, 0.15311652)]
document ID 9:[(5, 0.99907625)]


Let us check out the ```nth``` document in the result.

In [331]:
n=0

# nth document's topic distribution
print(ldamodel.get_document_topics(corpus_[n]))

# nth document's category
#print(categories[n])

# show the original document
print(" ".join(docs[n]))

[(1, 0.15732661), (2, 0.84181297)]
The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia 's registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that

## Visualization
We can further analyze our result through visualization.

In [332]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary)
lda_display

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.073563  0.083384       1        1  15.800656
1     -0.218472 -0.141623       2        1  13.798783
0      0.182705 -0.063473       3        1  12.099251
8     -0.095043  0.068417       4        1  10.953217
2      0.059580 -0.004450       5        1  10.871242
9      0.083865 -0.044533       6        1  10.039186
7      0.079952 -0.054476       7        1   8.054149
5      0.021961  0.088996       8        1   6.989165
4     -0.067337  0.022329       9        1   6.095543
3      0.026352  0.045429      10        1   5.298810, topic_info=          Term         Freq        Total Category  logprob  loglift
79      church   315.000000   315.000000  Default  30.0000  30.0000
459      state  1078.000000  1078.000000  Default  29.0000  29.0000
7524       **f   186.000000   186.000000  Default  28.0000  28.0000
1604      form   392.000000   392.000000  Default  27.0000  27.0000
472        tax   208.000000   208.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
1481     world    30.650616   527.694214  Topic10  -6.2768   0.0918
65        call    28.300856   409.748688  Topic10  -6.3565   0.2650
233      house    26.803783   456.675385  Topic10  -6.4109   0.1023
243   interest    25.822596   361.868591  Topic10  -6.4482   0.2977
117        day    26.119823   461.218262  Topic10  -6.4367   0.0665

[786 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
7666       1  0.034043     %
7666       2  0.765966     %
7666       3  0.059575     %
7666       4  0.017021     %
7666       5  0.008511     %
...      ...       ...   ...
988        8  0.184650  york
988        9  0.034934  york
988       10  0.054896  york
26174      5  0.907031   zen
26174      9  0.086384   zen

[2750 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 1, 9, 3, 10, 8, 6, 5, 4])

## Training with k = 20, alpha-beta-minimum_probability = 0.1

In [333]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           passes=50,
                                           alpha=0.1,                 # optional LDA hyperparameter alpha
                                           eta=0.1,                   # optional LDA hyperparameter beta
                                           minimum_probability=0.1    # optional the lower bound of the topic/word generative probability
                                          )

Check out the learned parameters.

In [334]:
# the top num_words of words for each topic (topic ID, the word generative probability for the topic).

topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.013*"form" + 0.010*"state" + 0.008*"hanover" + 0.007*"information" + 0.007*"dictionary" + 0.007*"text" + 0.006*"cell" + 0.006*"allotment" + 0.006*"h" + 0.005*"wave"')
(1, '0.005*"program" + 0.005*"music" + 0.004*"school" + 0.004*"vocational" + 0.003*"work" + 0.003*"man" + 0.003*"training" + 0.002*"concert" + 0.002*"negro" + 0.002*"orchestra"')
(2, '0.004*"area" + 0.004*"junior" + 0.004*"clay" + 0.003*"water" + 0.003*"class" + 0.003*"bridge" + 0.003*"work" + 0.003*"show" + 0.003*"place" + 0.003*"boat"')
(3, '0.009*"**f" + 0.005*"temperature" + 0.005*"anode" + 0.004*"vehicle" + 0.003*"show" + 0.003*"gas" + 0.003*"space" + 0.003*"polynomial" + 0.003*"arc" + 0.003*"operator"')
(4, '0.011*"line" + 0.006*"point" + 0.006*"g" + 0.005*"\\" + 0.004*"image" + 0.004*"mike" + 0.004*"meet" + 0.004*"parker" + 0.004*"secant" + 0.003*"pencil"')
(5, '0.022*"state" + 0.010*"business" + 0.007*"payment" + 0.006*"small" + 0.006*"government" + 0.005*"unite" + 0.005*"program" + 0.005*"act" + 0.005*"pro

In [335]:
# for each document, show the probabilities of topics which beyond the minimum_probability [(topic ID, probability)]

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(14, 0.9834266)]
document ID 1:[(14, 0.9981828)]
document ID 2:[(10, 0.8955735), (19, 0.10267401)]
document ID 3:[(17, 0.8511638), (19, 0.1469657)]
document ID 4:[(7, 0.9201405)]
document ID 5:[(17, 0.9930794)]
document ID 6:[(8, 0.62320775), (17, 0.15151457), (19, 0.1532559)]
document ID 7:[(12, 0.13427272), (16, 0.7053854), (19, 0.15845406)]
document ID 8:[(8, 0.9099019)]
document ID 9:[(1, 0.12082877), (14, 0.63314646), (19, 0.20550248)]


In [336]:
n=0

# nth document's topic distribution
print(ldamodel.get_document_topics(corpus_[n]))

# nth document's category
#print(categories[n])

# show the original document
print(" ".join(docs[n]))

[(14, 0.9834236)]
The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia 's registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislato

## Visualization
We can further analyze our result through visualization.

In [337]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary)
lda_display

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.191691 -0.199364       1        1  21.371176
12    -0.182307  0.088491       2        1  14.676245
8     -0.129723  0.076286       3        1  13.347310
14    -0.147109  0.092439       4        1  12.398607
2     -0.062931 -0.024754       5        1   4.853251
16    -0.024043 -0.013153       6        1   4.090940
1     -0.046005  0.005776       7        1   3.960166
7     -0.012987 -0.050704       8        1   3.586176
3      0.044950  0.060004       9        1   3.162863
15     0.013934 -0.050756      10        1   3.016553
13     0.015136 -0.008819      11        1   2.726590
6      0.012487 -0.031955      12        1   1.921366
10     0.078774 -0.061610      13        1   1.880597
9      0.073032 -0.051350      14        1   1.799293
5      0.046243  0.156386      15        1   1.706033
0      0.087731  0.060019      16        1   1.551800
4      0.085739 -0.045635      17        1   1.467302
17     0.064770  0.025474      18        1   1.409576
18     0.110340 -0.008204      19        1   1.062722
11     0.163660 -0.018570      20        1   0.011428, topic_info=           Term        Freq       Total Category  logprob  loglift
459       state  960.000000  960.000000  Default  30.0000  30.0000
4664        god  252.000000  252.000000  Default  29.0000  29.0000
284         man  866.000000  866.000000  Default  28.0000  28.0000
1604       form  352.000000  352.000000  Default  27.0000  27.0000
1840   business  220.000000  220.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
368     program    0.001185  361.399597  Topic20 -10.2984  -3.5513
3358       life    0.001185  512.234924  Topic20 -10.2984  -3.9001
1770    thought    0.001185  359.581390  Topic20 -10.2984  -3.5463
29565     viola    0.001185   16.035248  Topic20 -10.2985  -0.4361
740        high    0.001185  368.754303  Topic20 -10.2985  -3.5715

[1376 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
7666       3  0.265187         %
7666       4  0.343761         %
7666       5  0.009822         %
7666       9  0.137505         %
7666      11  0.009822         %
...      ...       ...       ...
1487      18  0.004391     young
18428     15  0.839587  yugoslav
26174      3  0.866230       zen
26174     10  0.082498       zen
20087      8  0.785882      zing

[5765 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 13, 9, 15, 3, 17, 2, 8, 4, 16, 14, 7, 11, 10, 6, 1, 5, 18, 19, 12])

## Training with k = 10, alpha-beta-minimum_probability = 0.6

In [338]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=10,
                                           id2word=dictionary,
                                           passes=50,
                                           alpha=0.6,                 # optional LDA hyperparameter alpha
                                           eta=0.6,                   # optional LDA hyperparameter beta
                                           minimum_probability=0.6    # optional the lower bound of the topic/word generative probability
                                          )

Check out the learned parameters.

In [339]:
# the top num_words of words for each topic (topic ID, the word generative probability for the topic).

topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.003*"line" + 0.002*"point" + 0.002*"phil" + 0.002*"g" + 0.002*"\\" + 0.002*"player" + 0.002*"palmer" + 0.001*"trial" + 0.001*"thyroid" + 0.001*"binomial"')
(1, '0.003*"**f" + 0.002*"feed" + 0.002*"temperature" + 0.002*"cell" + 0.002*"atom" + 0.002*"reaction" + 0.001*"show" + 0.001*"particle" + 0.001*"teeth" + 0.001*"small"')
(2, '0.002*"brown" + 0.001*"cooling" + 0.001*"rector" + 0.001*"maude" + 0.001*"sarah" + 0.001*"oersted" + 0.001*"sauce" + 0.001*"cup" + 0.001*"hino" + 0.001*"meat"')
(3, '0.003*"world" + 0.003*"life" + 0.002*"experience" + 0.002*"social" + 0.002*"century" + 0.002*"idea" + 0.002*"church" + 0.002*"people" + 0.002*"man" + 0.002*"human"')
(4, '0.003*"game" + 0.002*"run" + 0.002*"mantle" + 0.002*"hanover" + 0.002*"h" + 0.002*"age" + 0.002*"play" + 0.001*"league" + 0.001*"record" + 0.001*"onset"')
(5, '0.005*"man" + 0.005*"time" + 0.003*"long" + 0.003*"turn" + 0.003*"house" + 0.003*"men" + 0.003*"good" + 0.002*"\'ll" + 0.002*"place" + 0.002*"head"')
(6, '0.007*"st

In [340]:
# for each document, show the probabilities of topics which beyond the minimum_probability [(topic ID, probability)]

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(6, 0.9932481)]
document ID 1:[(6, 0.9937516)]
document ID 2:[(4, 0.6478836)]
document ID 3:[(4, 0.6543645)]
document ID 4:[(4, 0.7691414)]
document ID 5:[]
document ID 6:[(4, 0.699927)]
document ID 7:[(5, 0.72379273)]
document ID 8:[(6, 0.92681587)]
document ID 9:[(5, 0.8186041)]


In [341]:
n=0

# nth document's topic distribution
print(ldamodel.get_document_topics(corpus_[n]))

# nth document's category
#print(categories[n])

# show the original document
print(" ".join(docs[n]))

[(6, 0.9932483)]
The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia 's registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislator

## Visualization
We can further analyze our result through visualization.

In [342]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary)
lda_display

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.197679 -0.077204       1        1  41.820705
5     -0.140305  0.130367       2        1  32.548149
3     -0.060747 -0.049820       3        1  10.511654
4      0.036576  0.021979       4        1   3.501192
1      0.041251 -0.029213       5        1   3.407380
7      0.050392 -0.004489       6        1   2.525521
9      0.049018  0.000085       7        1   2.437381
0      0.069148  0.001929       8        1   2.038188
8      0.074837  0.000169       9        1   0.633951
2      0.077509  0.006196      10        1   0.575874, topic_info=              Term        Freq       Total Category  logprob  loglift
459          state  997.000000  997.000000  Default  30.0000  30.0000
343          point  320.000000  320.000000  Default  29.0000  29.0000
1651          line  244.000000  244.000000  Default  28.0000  28.0000
2067          show  362.000000  362.000000  Default  27.0000  27.0000
1481         world  460.000000  460.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
2217           cup    1.417793   31.732035  Topic10  -7.1310   2.0488
3714          heat    1.337478   47.779011  Topic10  -7.1893   1.5812
16084  conditioner    0.745862    8.669335  Topic10  -7.7733   2.7040
27467       cereal    0.705947    6.421957  Topic10  -7.8283   2.9491
3709           han    0.716060   38.013176  Topic10  -7.8141   1.1851

[622 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
989        1  0.004674    'd
989        2  0.990959    'd
990        1  0.060007   'll
990        2  0.922604   'll
990        4  0.011251   'll
...      ...       ...   ...
1484       2  0.783996  yard
1484       4  0.174221  yard
26174      1  0.053477   zen
26174      2  0.053477   zen
26174      3  0.909111   zen

[2206 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 4, 5, 2, 8, 10, 1, 9, 3])

## Training with k = 20, alpha-beta-minimum_probability = 0.6

In [343]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           passes=50,
                                           alpha=0.6,                 # optional LDA hyperparameter alpha
                                           eta=0.6,                   # optional LDA hyperparameter beta
                                           minimum_probability=0.6    # optional the lower bound of the topic/word generative probability
                                          )

Check out the learned parameters.

In [344]:
# the top num_words of words for each topic (topic ID, the word generative probability for the topic).

topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.001*"illusion" + 0.001*"collage" + 0.001*"surface" + 0.001*"depth" + 0.001*"braque" + 0.001*"picasso" + 0.001*"cubism" + 0.001*"literal" + 0.001*"flatness" + 0.000*"plane"')
(1, '0.002*"tone" + 0.001*"phonologic" + 0.001*"morphophonemics" + 0.000*"consonantal" + 0.000*"phonemic" + 0.000*"orthography" + 0.000*"linguist" + 0.000*"morphophonemic" + 0.000*"intonation" + 0.000*"tonal"')
(2, '0.001*"stem" + 0.001*"athabascan" + 0.001*"orchestra" + 0.001*"language" + 0.001*"concert" + 0.001*"sansom" + 0.001*"ballet" + 0.001*"season" + 0.001*"swadesh" + 0.000*"larkin"')
(3, '0.002*"soil" + 0.002*"allotment" + 0.001*"surface" + 0.001*"fogg" + 0.001*"detergent" + 0.001*"agent" + 0.001*"machine" + 0.001*"particle" + 0.001*"pansy" + 0.001*"quack"')
(4, '0.001*"serum" + 0.001*"activity" + 0.001*"antibody" + 0.001*"cell" + 0.001*"saline" + 0.001*"fraction" + 0.001*"sample" + 0.001*"buffer" + 0.001*"dialysis" + 0.001*"gradient"')
(5, '0.002*"jess" + 0.002*"curt" + 0.001*"model" + 0.001*"magnum

In [345]:
# for each document, show the probabilities of topics which beyond the minimum_probability [(topic ID, probability)]

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(17, 0.986452)]
document ID 1:[(17, 0.98755026)]
document ID 2:[(8, 0.85533476)]
document ID 3:[(8, 0.7886676)]
document ID 4:[(8, 0.93819034)]
document ID 5:[]
document ID 6:[(8, 0.8390402)]
document ID 7:[]
document ID 8:[(17, 0.92792994)]
document ID 9:[]


In [346]:
n=0

# nth document's topic distribution
print(ldamodel.get_document_topics(corpus_[n]))

# nth document's category
#print(categories[n])

# show the original document
print(" ".join(docs[n]))

[(17, 0.9864525)]
The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia 's registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislato

## Visualization
We can further analyze our result through visualization.

In [347]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary)
lda_display

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17     0.233181  0.102402       1        1  43.449768
18     0.172034 -0.023405       2        1  24.307104
8      0.149797 -0.124322       3        1  22.018812
11    -0.027205  0.005058       4        1   1.458454
15    -0.020506  0.021085       5        1   1.431626
3     -0.034717  0.003501       6        1   1.022371
9     -0.034258  0.010314       7        1   0.976115
13    -0.037783  0.004242       8        1   0.844357
19    -0.035475  0.001073       9        1   0.776356
6     -0.032571  0.000913      10        1   0.702262
7     -0.037812 -0.002560      11        1   0.659514
2     -0.035939  0.000422      12        1   0.551480
10    -0.031277  0.004526      13        1   0.342219
4     -0.032380  0.003499      14        1   0.306718
5     -0.033541 -0.001633      15        1   0.247746
12    -0.030781 -0.002363      16        1   0.245852
0     -0.032093 -0.000939      17        1   0.206936
16    -0.032936 -0.000717      18        1   0.156155
1     -0.033013 -0.001063      19        1   0.150054
14    -0.032723 -0.000034      20        1   0.146103, topic_info=             Term        Freq       Total Category  logprob  loglift
1604         form  295.000000  295.000000  Default  30.0000  30.0000
343         point  330.000000  330.000000  Default  29.0000  29.0000
459         state  996.000000  996.000000  Default  28.0000  28.0000
1651         line  259.000000  259.000000  Default  27.0000  27.0000
757   information  140.000000  140.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
194      friendly    0.065919   33.963890  Topic20  -8.8278   0.2840
3626         core    0.064386   28.510441  Topic20  -8.8514   0.4355
8372    replenish    0.055829    5.621246  Topic20  -8.9940   1.9166
2822       collar    0.057637   10.876760  Topic20  -8.9621   1.2884
4610      contact    0.058441   63.381187  Topic20  -8.9482  -0.4603

[1038 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
989        1  0.004895     'd
989        2  0.107683     'd
989        3  0.881044     'd
990        1  0.069587    'll
990        2  0.154637    'll
...      ...       ...    ...
26174      1  0.187464    zen
26174      2  0.562392    zen
26174     18  0.187464    zen
26175      1  0.392036  zendo
26175      2  0.392036  zendo

[2749 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 19, 9, 12, 16, 4, 10, 14, 20, 7, 8, 3, 11, 5, 6, 13, 1, 17, 2, 15])